<a href="https://colab.research.google.com/github/SpectraGbes/Spectragbes/blob/main/RAD_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu sentence-transformers transformers langchain==0.1.0 accelerate

In [2]:
!pip install python-docx

In [3]:
from google.colab import files
import docx

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print("Uploaded:", file_name)

doc = docx.Document(file_name)
text = "\n".join(p.text for p in doc.paragraphs)
print(text[:500])  # preview

Saving Water_Treatment_Policy.docx to Water_Treatment_Policy (1).docx
Uploaded: Water_Treatment_Policy (1).docx
WATER TREATMENT OPERATIONS POLICY AND PROCEDURES
Updated: January 2026
SECTION 1 — DAILY WATER QUALITY MONITORING
1. Operators must check pH levels every 2 hours during all production shifts.
2. The acceptable pH range is 6.8 – 7.4.
3. If pH goes below 6.5 or above 7.8, the operator must adjust chemical dosing immediately and inform the shift supervisor.
4. Turbidity must remain below 1 NTU at all times for treated water.
5. Chlorine residual must stay between 0.2 – 0.5 mg/L throughout productio


In [5]:
!pip install langchain-text-splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = splitter.split_text(text)
print("Total chunks:", len(chunks))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.2/319.2 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsmith-0.0.87
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.20 requires langchain-core<0.2,>=0.1.21, but you have langchain-core 1.2.9 which is incompatible.
langchain-community 0.0.20 requires langsmith<0.1,>=0.0.83, but you have langsmith 0.6.9 which is incompatible.
langchain 0.1.0 requires langchain-core<0.2,>=0.1.7, but you have langchain

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(chunks)
embeddings = np.array(embeddings).astype("float32")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
import faiss

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print("Vector DB ready. Total chunks:", index.ntotal)

Vector DB ready. Total chunks: 7


In [8]:
def retrieve(query, top_k=3):
    q_embed = model.encode([query]).astype("float32")
    distances, indices = index.search(q_embed, top_k)
    results = [chunks[i] for i in indices[0]]
    return results

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

In [23]:
def retrieve(query, top_k=3):
    q_embed = model.encode([query]).astype("float32")
    distances, indices = index.search(q_embed, top_k)
    retrieved_chunks = [chunks[i] for i in indices[0]]
    return retrieved_chunks

In [24]:
def generate_answer(query):
    retrieved = retrieve(query)

    context = "\n\n".join(retrieved)

    prompt = f"""
Use ONLY the following context to answer the question.
If the answer is not found in the context, say "I don't know."

Context:
{context}

Question: {query}
Answer:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)
    outputs = llm.generate(
        inputs["input_ids"],
        max_new_tokens=250,
        temperature=0.2
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [14]:
def generate_answer(query):
    retrieved = retrieve(query)
    context = "\n\n".join(retrieved)
    answer = generate_answer_gpt2(query, context)
    return answer

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "gpt2"  # replace with your model

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer_gpt2 = AutoTokenizer.from_pretrained(MODEL_NAME)
llm_gpt2 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # torch_dtype=torch.float16,  # uncomment if on GPU that supports FP16
    # device_map="auto",          # uncomment if using accelerate
)
llm_gpt2.to(device)
llm_gpt2.eval()

# Ensure padding is defined
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token
llm_gpt2.config.pad_token_id = tokenizer_gpt2.eos_token_id

def generate_answer_gpt2(query: str, context: str = "", max_new_tokens: int = 128) -> str:
    """
    Generate a concise answer with guardrails to avoid long stalls.
    """
    prompt = (
        "You are a helpful assistant. Answer succinctly based on the context.\n\n"
        f"Context:\n{context[:4000]}\n\n"  # hard cap context length
        f"Question: {query}\nAnswer:"
    )

    inputs = tokenizer_gpt2(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024,
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = llm_gpt2.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            min_new_tokens=0,
            do_sample=False,
            num_beams=1,
            temperature=1.0,
            top_p=1.0,
            use_cache=True,
            pad_token_id=tokenizer_gpt2.eos_token_id,
            eos_token_id=tokenizer_gpt2.eos_token_id
        )

    text = tokenizer_gpt2.decode(output_ids[0], skip_special_tokens=True)
    # Trim everything before "Answer:" if you’re formatting the prompt
    if "Answer:" in text:
        text = text.split("Answer:", 1)[-1].strip()
    return text


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [15]:
import signal

class TimeoutError(Exception):
    pass

def _timeout_handler(signum, frame):
    raise TimeoutError

signal.signal(signal.SIGALRM, _timeout_handler)
signal.alarm(60)  # Increased timeout to 60 seconds

query = "What does this document say about water treatment policy?"

try:
    answer = generate_answer(query)
finally:
    signal.alarm(0)

print("Answer from GPT2 model:")
print(answer)

Answer from GPT2 model:
Water treatment policy is a set of guidelines that govern the use of water in the workplace. The following are the guidelines:

1. Water treatment must be done in a safe and sanitary environment.

2. Water treatment must be done in a clean and sanitary environment.

3. Water treatment must be done in a clean and sanitary environment.

4. Water treatment must be done in a clean and sanitary environment.

5. Water treatment must be done in a clean and sanitary environment.

6. Water treatment must be done in a clean and sanitary environment.


In [21]:
query = "Explain in detail water treatment policy?"
print("Answer from GPT2 model:")
print(answer)

Answer from GPT2 model:
Water treatment policy is a set of guidelines that govern the use of water in the workplace. The following are the guidelines:

1. Water treatment must be done in a safe and sanitary environment.

2. Water treatment must be done in a clean and sanitary environment.

3. Water treatment must be done in a clean and sanitary environment.

4. Water treatment must be done in a clean and sanitary environment.

5. Water treatment must be done in a clean and sanitary environment.

6. Water treatment must be done in a clean and sanitary environment.


In [16]:
pip install gradio

In [26]:
import gradio as gr

def chat_fn(message, history):
    answer = generate_answer(message)
    history.append((message, answer))
    return history, history

chatbot = gr.ChatInterface(
    fn=chat_fn,
    title="Henry's RAG AI Assistant",
    description="Ask questions based on the uploaded document."
)

chatbot.launch()

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2fcc418a5095dbaf17.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
